In [31]:
import re
import os.path as path

import nltk
import treetaggerwrapper
from gensim.models.phrases import Phrases

In [32]:
def load_stopwords(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        data = f.read()
        return set(data.split('\n'))

tokenizer = nltk.data.load('tokenizers/punkt/german.pickle')
stopwords = load_stopwords(path.join('data', 'Stopwords.txt'))
tagger = treetaggerwrapper.TreeTagger(TAGLANG='de')
allowed_tags = set(['NN', 'NE', 'VVPP', 'VVFIN','ADJD', 'ADJA'])

def clean_lemma(lemma):
    if '|' in lemma:
        clean = lemma.split('|')
        return clean[0].strip()
    else:
        return lemma

def sentence_to_words(sentence):
    letters_only = re.sub(r'[^a-zäöüA-ZÖÄÜß]', ' ', sentence)
    tagged = tagger.tag_text(letters_only)
    tags = treetaggerwrapper.make_tags(tagged, exclude_nottags=True)
    words = [clean_lemma(lemma.lower()) for word, tag, lemma in tags 
             if tag in allowed_tags and word not in stopwords]
    return words

# def sentence_to_words(sentence):
#     letters_only = re.sub(r'[^a-zäöüA-ZÖÄÜß]', ' ', sentence)
#     words = letters_only.lower().split()
#     words = [w for w in words if w not in stopwords]
#     return words

def document_to_sentences(document):
    raw_sentences = tokenizer.tokenize(document.strip())
    sentences = []
    for sentence in raw_sentences:
        sentences.append(sentence_to_words(sentence))
    return sentences

class StreamCorpus(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for document in open(self.filename, encoding='UTF-8'):
            sentences = document_to_sentences(document)
            for sentence in sentences:
                yield sentence

In [33]:
corpus_file = path.join('data', 'Corpus.txt')
corpus = StreamCorpus(corpus_file)
bigram = Phrases(corpus)

In [34]:
query1 = """Ein britischer Islamist mit dem Spitznamen Dschihadi John war in mehreren Enthauptungsvideos 
           des Islamischen Staats zu sehen. Nun haben die USA ihn bei einem Luftangriff in Syrien ins Visier 
           genommen. Ob der Mann starb, ist noch unklar. Angela Merkel und Horst Seehofer haben den Fall schon kommentiert."""
query2 = """Während man sich in Berlin fast nur über die Flüchtlingskrise unterhielt und in London über David Camerons 
            Forderungskatalog an die EU, verbrachte ich das vergangene Wochenende auf einem englischen Schloss. 
            Das Thema war, ob man den Euro retten oder aufgeben sollte. Die Atmosphäre der Veranstaltung war einer 
            Edgard-Wallace-Verfilmung würdig. Es gab meines Wissens nach keinen Mord, aber man hatte zuweilen den Eindruck, 
            dass dazu nicht viel fehlen würde. Wie bei den Edgar-Wallace-Filmen der sechziger Jahre waren viele der 
            Hauptakteure Deutsche. Und um Deutschland ging es am Ende auch in jeder einzelnen Debatte, direkt oder indirekt."""
sentences1 = document_to_sentences(query1)
sentences2 = document_to_sentences(query2)
for sentence in sentences1:
    print(bigram[sentence])

for sentence in sentences2:
    print(bigram[sentence])

['britisch', 'islamist', 'spitzname', 'dschihadi', 'john', 'enthauptungsvideos', 'islamisch_staat']
['usa', 'luftangriff_syrien', 'visier']
['mann', 'sterben', 'unklar']
['angela_merkel', 'horst_seehofer', 'fall', 'kommentieren']
['berlin', 'flüchtlingskrise', 'unterhalten', 'london', 'david_camerons', 'forderungskatalog', 'eu', 'verbringen', 'wochenende', 'englisch', 'schloss']
['thema', 'euro']
['atmosphäre', 'veranstaltung', 'edgard', 'wallace_verfilmung', 'würdig']
['wissen', 'mord', 'eindruck']
['edgar_wallace', 'film', 'sechziger_jahr', 'hauptakteur', 'deutsche']
['deutschland', 'gehen', 'ende', 'einzeln', 'debatte', 'indirekt']
